# 대출 이자 계산 알고리즘

## 대출 상환 방식
- 원금균등상환
- 원리금균등상환
- 만기일시상환

In [5]:
import datetime
import pandas as pd

### 원금균등상환

In [16]:
def calculate_emi(principal, rate, periods):
    emi = principal * rate * (1 + rate) ** periods / ((1 + rate) ** periods - 1)
    return emi

def amortization_schedule(principal, annual_rate, periods, frequency='monthly'):
    if frequency == 'monthly':
        period_rate = annual_rate / 12
        total_periods = periods
    elif frequency == 'biweekly':
        period_rate = annual_rate / 26
        total_periods = periods * 12 // 26
    elif frequency == 'weekly':
        period_rate = annual_rate / 52
        total_periods = periods * 12 // 52
    else:
        raise ValueError("Invalid frequency. Choose from 'monthly', 'biweekly', or 'weekly'.")
    
    emi = calculate_emi(principal, period_rate, total_periods)
    
    schedule = []
    outstanding_principal = principal
    
    for period in range(1, total_periods + 1):
        interest_payment = outstanding_principal * period_rate
        principal_payment = emi - interest_payment
        outstanding_principal -= principal_payment
        
        schedule.append({
            'period': period,
            'emi': round(emi, 2),
            'principal_payment': round(principal_payment, 2),
            'interest_payment': round(interest_payment, 2),
            'outstanding_principal': round(outstanding_principal, 2)
        })
        
    return schedule

# 예시: 대출 원금 10,000, 연이율 5%, 기간 12개월, 2주마다 상환
principal = 10000
annual_rate = 0.05
periods = 12
frequency = 'biweekly'

schedule = amortization_schedule(principal, annual_rate, periods, frequency)
for payment in schedule:
    print(payment)


{'period': 1, 'emi': 2011.55, 'principal_payment': 1992.32, 'interest_payment': 19.23, 'outstanding_principal': 8007.68}
{'period': 2, 'emi': 2011.55, 'principal_payment': 1996.15, 'interest_payment': 15.4, 'outstanding_principal': 6011.52}
{'period': 3, 'emi': 2011.55, 'principal_payment': 1999.99, 'interest_payment': 11.56, 'outstanding_principal': 4011.53}
{'period': 4, 'emi': 2011.55, 'principal_payment': 2003.84, 'interest_payment': 7.71, 'outstanding_principal': 2007.69}
{'period': 5, 'emi': 2011.55, 'principal_payment': 2007.69, 'interest_payment': 3.86, 'outstanding_principal': -0.0}
